# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("orientdb", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
client.db_create( "gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
client.db_open( "gods","admin","admin",pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY) 

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
client.command("CREATE VERTEX V CONTENT " + str({"name": "Zeus", "symbol": "thunder"}))

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [7]:
for god in client.command("SELECT * FROM V") :
    print(god)

{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [8]:
new_content=[
    {"name":"Héra", "symbol":"tiara"},
    {"name":"Poséidon", "symbol":"trident"},
    {"name":"Athena", "symbol":"helmet"},
    {"name":"Arès", "symbol":"weapons"}
]
for god in new_content : 
    client.command("CREATE VERTEX V CONTENT "+str(god))

**Q:** Display all vertices with name = `Arès`

In [9]:
for god in client.command("SELECT * FROM V WHERE name LIKE 'Arès'") :
    print(god)

{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
client.command(
    "CREATE EDGE E FROM "
    "(SELECT FROM V WHERE name = 'Zeus') "
    "TO (SELECT FROM V WHERE name = 'Poséidon') "
    "CONTENT " + str({"kind": "sibling"})
)

**Q:** Redisplay all vertices, discuss.

In [11]:
for god in client.command("SELECT * FROM V") :
    print(god)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x76aae6cb8820>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x76aae43c65c0>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [12]:
for edg in client.command("SELECT * FROM E") :
    print(edg)

{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae430f4c0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae6c73820>},'version':1,'rid':'#25:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [13]:
new_edges=[
    ["Zeus","Héra","sibling"],
    ["Zeus","Arès","father"],
    ["Zeus","Athena","father"],
    ["Héra","Arès","mother"],
    ["Héra","Zeus","sibling"],
    ["Poséidon","Zeus","sibling"]
]
for edge in new_edges:
    sql_query = (
        f"CREATE EDGE E FROM "
        f"(SELECT FROM V WHERE name = '{edge[0]}') "
        f"TO (SELECT FROM V WHERE name = '{edge[1]}') "
        f"CONTENT {{\"kind\": '{edge[2]}'}}"
    )
    client.command(sql_query)
    print(f"Lien créé : {edge[0]} --({edge[2]})--> {edge[1]}")
for edg in client.command("SELECT * FROM E") :
    print(edg)

Lien créé : Zeus --(sibling)--> Héra
Lien créé : Zeus --(father)--> Arès
Lien créé : Zeus --(father)--> Athena
Lien créé : Héra --(mother)--> Arès
Lien créé : Héra --(sibling)--> Zeus
Lien créé : Poséidon --(sibling)--> Zeus
{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae430e2c0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43c73d0>},'version':1,'rid':'#25:0'}
{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43c7520>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43c7a90>},'version':1,'rid':'#26:0'}
{'@E':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43c7f40>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43c6590>},'version':1,'rid':'#27:0'}
{'@E':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43c7610>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43c6ad0>},'version':1,'rid':'#28:0'}
{'@E':{'kind': 'mother', 

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [14]:
for vertice in client.command("SELECT EXPAND(out()) FROM V WHERE name='Zeus'") : 
    print(f"Nom: {vertice.name}, Symbole: {vertice.symbol}")
    #print(vertice)

Nom: Poséidon, Symbole: trident
Nom: Héra, Symbole: tiara
Nom: Arès, Symbole: weapons
Nom: Athena, Symbole: helmet


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [15]:
for vert in client.command("SELECT EXPAND(in) FROM E WHERE kind='father'") : 
    print(vert)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x76aae430e2c0>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x76aae43c7190>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [16]:
query="SELECT FROM V WHERE outE() IN (SELECT FROM E WHERE kind = 'mother')"

for vert in client.command(query) : 
    print(vert.name)

Héra


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [17]:
query="SELECT EXPAND(in) FROM E WHERE kind = 'sibling' AND out IN (SELECT FROM V WHERE name = 'Zeus')"

for vert in client.command(query) : 
    print(vert.name)

Poséidon
Héra


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [18]:
client.connect("root", ROOT_PASSWORD)
client.db_create( "eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )
client.db_open( "eCommerce","admin","admin",pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY) 
print("base créée")

base créée


In [19]:
client.command("CREATE CLASS Person EXTENDS V")
client.command("CREATE CLASS Product EXTENDS V")
client.command("CREATE CLASS Bought EXTENDS E")
print("Classes dans la base eCommerce :")
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

Classes dans la base eCommerce :
{{'name': 'OSequence'},'version':0,'rid':'#-2:0'}
{{'name': 'V'},'version':0,'rid':'#-2:1'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:2'}
{{'name': 'E'},'version':0,'rid':'#-2:3'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:4'}
{{'name': 'Person'},'version':0,'rid':'#-2:5'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:6'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:7'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:8'}
{{'name': 'OUser'},'version':0,'rid':'#-2:9'}
{{'name': 'ORole'},'version':0,'rid':'#-2:10'}
{{'name': 'Product'},'version':0,'rid':'#-2:11'}
{{'name': 'Bought'},'version':0,'rid':'#-2:12'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [20]:
products=["spaghetti","bolognese sauce","cheese","apple"]
for product_name in products:
    client.command(f"CREATE VERTEX Product CONTENT {{'name': '{product_name}'}}")
print("Produits insérés avec succès.")
for person in client.command("SELECT * FROM Product") :
    print(person)

Produits insérés avec succès.
{'@Product':{'name': 'spaghetti'},'version':1,'rid':'#57:0'}
{'@Product':{'name': 'bolognese sauce'},'version':1,'rid':'#58:0'}
{'@Product':{'name': 'cheese'},'version':1,'rid':'#59:0'}
{'@Product':{'name': 'apple'},'version':1,'rid':'#60:0'}


**Q:** Create the following people: `peter`, `meredith`.

In [21]:
peoples=["peter","meredith"]
for person_name in peoples:
    client.command(f"CREATE VERTEX Person CONTENT {{'nickname': '{person_name}'}}")
print("Personnes insérés avec succès.")
for person in client.command("SELECT * FROM Person") :
    print(person)

Personnes insérés avec succès.
{'@Person':{'nickname': 'peter'},'version':1,'rid':'#41:0'}
{'@Person':{'nickname': 'meredith'},'version':1,'rid':'#42:0'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [22]:
new_edges = [
    ["peter", "spaghetti", "2016-01-20"],
    ["peter", "cheese", "2016-01-20"],
    ["meredith", "cheese", "2016-01-22"],
    ["meredith", "apple", "2016-01-22"],
    ["meredith", "bolognese sauce", "2016-01-22"],
    ["peter", "spaghetti", "2016-01-27"],
    ["peter", "bolognese sauce", "2016-01-27"]
]
for edge in new_edges:
    sql_query = (
        f"CREATE EDGE Bought FROM "
        f"(SELECT FROM Person WHERE nickname = '{edge[0]}') "
        f"TO (SELECT FROM Product WHERE name = '{edge[1]}') "
        f"SET date = '{edge[2]}'"
    )
    client.command(sql_query)
    print(f"Lien créé : {edge[0]} --({edge[2]})--> {edge[1]}")
for edg in client.command("SELECT * FROM Bought") :
    print(edg)

Lien créé : peter --(2016-01-20)--> spaghetti
Lien créé : peter --(2016-01-20)--> cheese
Lien créé : meredith --(2016-01-22)--> cheese
Lien créé : meredith --(2016-01-22)--> apple
Lien créé : meredith --(2016-01-22)--> bolognese sauce
Lien créé : peter --(2016-01-27)--> spaghetti
Lien créé : peter --(2016-01-27)--> bolognese sauce
{'@Bought':{'date': '2016-01-20', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43f0b80>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43f1270>},'version':1,'rid':'#73:0'}
{'@Bought':{'date': '2016-01-20', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43f15a0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43f1090>},'version':1,'rid':'#74:0'}
{'@Bought':{'date': '2016-01-22', 'out': <pyorient.otypes.OrientRecordLink object at 0x76aae43f1450>, 'in': <pyorient.otypes.OrientRecordLink object at 0x76aae43f0b20>},'version':1,'rid':'#75:0'}
{'@Bought':{'date': '2016-01-22', 'out': <pyorient.otypes.OrientRecordLink obje

**Q:** Who bought Bolognese sauce?

In [23]:
query="SELECT EXPAND(out) FROM Bought WHERE in IN (SELECT FROM Product WHERE name = 'bolognese sauce')"

for person in client.command(query) : 
    print(person.nickname)

meredith
peter


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [24]:
query = "SELECT EXPAND(in('Bought').out('Bought')) FROM Product WHERE name = 'bolognese sauce'"

for product in client.command(query):
    if product.name != 'bolognese sauce':
        print(f"Produit acheté avec : {product.name}")

Produit acheté avec : cheese
Produit acheté avec : apple
Produit acheté avec : spaghetti
Produit acheté avec : cheese
Produit acheté avec : spaghetti


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.